In [11]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import seaborn as sns
sns.set(style='whitegrid',
        rc={'lines.linewidth': 2.5,
        'figure.figsize': (10, 8),
        'text.usetex': False,
        # 'font.family': 'sans-serif',
        # 'font.sans-serif': 'Optima LT Std',
        })

from pandas import set_option
set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from scipy.stats import truncnorm

filename = 'BDOShoham.csv'
training_data = pd.read_csv(filename)
training_data


# Flow Patterns
# 1=DB  2=SS   3=SW
# 4=A 5=I 7=B
flowpatterns_colors = ['#F4D03F', '#F5B041','#DC7633','#6E2C00','#1B4F72','#2E86C1']

flowpatterns_labels = ['DB', 'SS', 'SW', 'A', 'I', 'B']

correct_FlowPattern_labels = training_data['FlowPattern'].values

feature_vectors = training_data.drop(['FlowPattern'], axis=1)
feature_vectors.describe()


scaler = preprocessing.StandardScaler().fit(feature_vectors)
scaled_features = scaler.transform(feature_vectors)

X_train, X_test, y_train, y_test = train_test_split(
                                scaled_features, correct_FlowPattern_labels, test_size=0.2, random_state=42)


##APPLY OPTIMIZATION HERE
clf = MLPClassifier(solver='lbfgs', alpha=.1,
                    hidden_layer_sizes=(20,10,20))
#clf = MLPClassifier(solver='lbfgs', alpha=.01, activation ='tanh',
                        #hidden_layer_sizes=(130,145,150)) 


clf.fit(X_train,y_train)
conf_te = confusion_matrix(y_test, clf.predict(X_test))

def accuracy(conf):
    total_correct = 0.
    nb_classes = conf.shape[0]
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
    acc = total_correct/sum(sum(conf))
    return acc

print('Predicted accuracy: %.3f%%' % (100*accuracy(conf_te),))

predicted_labels = clf.predict(X_test)

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels]+[5]) # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print ("    " + empty_cell,)
    for label in labels:
        print ("%{0}s".format(columnwidth) % label,)
    print
    # Print rows
    for i, label1 in enumerate(labels):
        print ("    %{0}s".format(columnwidth) % label1,)
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print (cell,)
        print

conf = confusion_matrix(y_test, predicted_labels)
# then print it in a pretty way
#print_cm(conf, flowpatterns_labels)

print('Flow Pattern classification accuracy = %f' % accuracy(conf))

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
print(f'Training Model Accuracy: {clf.score(X_train,y_train)}')
print ('\n Accuracy:', accuracy_score(y_test, predicted_labels))
print ('F1 score:', f1_score(y_test, predicted_labels,average='weighted'))
print ('Recall:', recall_score(y_test, predicted_labels,
                              average='weighted'))
print ('Precision:', precision_score(y_test, predicted_labels,
                                    average='weighted'))
print ('\n clasification report:\n', classification_report(y_test, predicted_labels))
print ('\n confusion matrix:\n',confusion_matrix(y_test, predicted_labels))



Predicted accuracy: 85.463%
Flow Pattern classification accuracy = 0.854626
Training Model Accuracy: 0.8526431718061674

 Accuracy: 0.8546255506607929
F1 score: 0.8555434097863529
Recall: 0.8546255506607929
Precision: 0.8581531392970264

 clasification report:
               precision    recall  f1-score   support

           1       0.90      0.87      0.89       102
           2       0.67      0.52      0.58        27
           3       0.70      0.77      0.73       192
           4       0.82      0.88      0.85       200
           5       0.92      0.89      0.90       593
           7       0.90      0.90      0.90        21

    accuracy                           0.85      1135
   macro avg       0.82      0.80      0.81      1135
weighted avg       0.86      0.85      0.86      1135


 confusion matrix:
 [[ 89   0   3   0  10   0]
 [  0  14   6   0   7   0]
 [  0   6 147  25  14   0]
 [  0   0  14 175  11   0]
 [ 10   1  41  13 526   2]
 [  0   0   0   0   2  19]]


/Users/eslemus/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [12]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=100)

parameter_space = {
    'hidden_layer_sizes': [(130,145,150), (145, 150, 130)],
    'activation': [ 'relu', 'tanh'],
    'solver': ['sgd', 'lbfgs'],
    'alpha': [0.1, .01],
}

In [13]:
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space)
clf.fit(X_train,y_train)

/Users/eslemus/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/eslemus/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/eslemus/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/eslemus/opt/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

GridSearchCV(cv=None, error_score=nan,
             estimator=MLPClassifier(activation='relu', alpha=0.0001,
                                     batch_size='auto', beta_1=0.9,
                                     beta_2=0.999, early_stopping=False,
                                     epsilon=1e-08, hidden_layer_sizes=(100,),
                                     learning_rate='constant',
                                     learning_rate_init=0.001, max_fun=15000,
                                     max_iter=200, momentum=0.9,
                                     n_iter_no_change=10,
                                     nesterovs_momentum=True, power_t=0.5,
                                     random_state=None, shuffle=True,
                                     solver='adam', tol=0.0001,
                                     validation_fraction=0.1, verbose=False,
                                     warm_start=False),
             iid='deprecated', n_jobs=None,
             param_gr

In [14]:
# Best parameter set
from sklearn.neural_network import MLPClassifier
print('Best parameters found:\n', clf.best_params_)

# All results 

mean = clf.cv_results_['mean_test_score']
std = clf.cv_results_['std_test_score']

for means, stds, params in zip(mean, std, ['params']):
    print("%0.3f (+/-%0.03f) for %r" % (means, stds * 2, params))

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (130, 145, 150), 'solver': 'lbfgs'}
0.771 (+/-0.034) for 'params'


In [15]:
conf_te = confusion_matrix(y_test, clf.predict(X_test))

def accuracy(conf):
    total_correct = 0.
    nb_classes = conf.shape[0]
    for i in np.arange(0,nb_classes):
        total_correct += conf[i][i]
    acc = total_correct/sum(sum(conf))
    return acc

print('Predicted accuracy: %.3f%%' % (100*accuracy(conf_te),))

predicted_labels = clf.predict(X_test)

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels]+[5]) # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print ("    " + empty_cell,)
    for label in labels:
        print ("%{0}s".format(columnwidth) % label,)
    print
    # Print rows
    for i, label1 in enumerate(labels):
        print ("    %{0}s".format(columnwidth) % label1,)
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print (cell,)
        print

conf = confusion_matrix(y_test, predicted_labels)
# then print it in a pretty way
#print_cm(conf, flowpatterns_labels)

print('Flow Pattern classification accuracy = %f' % accuracy(conf))

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
print(f'\n Training Model Accuracy: {clf.score(X_train,y_train)}')
print ('\n Accuracy:', accuracy_score(y_test, predicted_labels))
print ('F1 score:', f1_score(y_test, predicted_labels,average='weighted'))
print ('Recall:', recall_score(y_test, predicted_labels,
                              average='weighted'))
print ('Precision:', precision_score(y_test, predicted_labels,
                                    average='weighted'))
print ('\n clasification report:\n', classification_report(y_test, predicted_labels))
print ('\n confusion matrix:\n',confusion_matrix(y_test, predicted_labels))



Predicted accuracy: 94.361%
Flow Pattern classification accuracy = 0.943612

 Training Model Accuracy: 0.9704845814977974

 Accuracy: 0.9436123348017621
F1 score: 0.94380507008474
Recall: 0.9436123348017621
Precision: 0.9446625734483298

 clasification report:
               precision    recall  f1-score   support

           1       0.84      0.91      0.87       102
           2       0.85      0.81      0.83        27
           3       0.96      0.90      0.93       192
           4       0.93      0.95      0.94       200
           5       0.97      0.96      0.97       593
           7       1.00      0.95      0.98        21

    accuracy                           0.94      1135
   macro avg       0.92      0.92      0.92      1135
weighted avg       0.94      0.94      0.94      1135


 confusion matrix:
 [[ 93   0   0   0   9   0]
 [  0  22   5   0   0   0]
 [  2   3 173  11   3   0]
 [  0   0   3 191   6   0]
 [ 16   1   0   4 572   0]
 [  0   0   0   0   1  20]]


Multilayer perceptrons are multiple layers of neurons interconnected in a feed-forward propogation that is applied by an activation function. Deep Learning is a more accurate algorithm becuase of the automatic feature engineering. Therefore, parameters are left to us to optomize to increase performance. 

For parameter optimization Grid Search function is used with Multilayer Perceptons as the algorithm estimator. The definition for searching for the best parameters is the activation and the solver, as well as, specifying the hidden layers. Within Grid Search function many other parameters can be optimized, such as epochs. For simplicity purposes, the most important parameters were optomized in this assignment.

By optimizing MLP, the training data increases its accuracy from 84% to 97% . 
Similarly for the test prediction accuracy, it increases from 84% to 94%. The increase percentage amounts to more than 10% per training and testing data. 

Multilayer perceptrons are able to increase highly with its accuracy scores meaning that Deep Learning-MLP evaluates to the most optimal performance that we've seen so far in class, where automatically learning features at multiple levels of abstraction allows the model to learn complex functions.

By Estefany Lemus